In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import vtk

plt.style.use('default')

In [ ]:
position_file_location = 'rank_0_positions.txt'

with open(position_file_location, 'r') as file:
    
    file.readline()  # Skip the line with the number of nodes
    file.readline()  # Skip the min/max lines
    file.readline()  # Skip the min/max lines
    file.readline()  # Skip the empty line
        
    positions = []
    labels = []
    types = []
        
    for line in file:
        # Skip lines that start with "Maximum" or "Minimum" or any comment lines
        if line.startswith("Maximum") or line.startswith("Minimum") or line.startswith("#"):
            continue
                
        # Split the line by spaces
        parts = line.split()
            
        # Ensure the line has the correct number of parts
        if len(parts) >= 6:
            x, y, z = map(float, parts[1:4])
            area = parts[4]
            neuron_type = parts[5]
                    
            positions.append([x, y, z])
            labels.append(area)
            types.append(neuron_type)
    
    positions = np.array(positions)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

unique_areas = list(set(labels))
colors = plt.cm.get_cmap('tab10', len(unique_areas))
area_color_map = {area: colors(i) for i, area in enumerate(unique_areas)}

node_colors = [area_color_map[area] for area in labels]

ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c=node_colors, s=1)

plt.show()

In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv('rank_0_positions.txt', skiprows= 8, delimiter=' ', names=column_headers_pos)
df_pos.head()


In [ ]:
column_headers_network = ['target rank', 'target ID', 'source rank', 'source ID', 'weigth']

df_nw = pd.read_csv('rank_0_step_40000_in_network.txt', skiprows= 5, delimiter=r'\s+', names=column_headers_network)
df_nw.head()


In [ ]:
graph = vtk.vtkMutableUndirectedGraph()

positions = vtk.vtkPoints()

for ix, id in enumerate(df_pos['local id']):
    id = graph.AddVertex()
    
    positions.InsertNextPoint(df_pos[ix]['pos x'], df_pos[ix]['pos y'], df_pos[ix]['pos z'] )
    
